In [1]:
import pandas as pd
import numpy as np
import os
import ast

In [2]:
event_df = pd.read_csv('data/all_events.csv')
player_df = pd.read_csv('data/all_players.csv')

C:\Users\samue\AppData\Local\Temp\ipykernel_5872\726313840.py:1: DtypeWarning: Columns (37,38,39,41,45) have mixed types. Specify dtype option on import or set low_memory=False.
  event_df = pd.read_csv('data/all_events.csv')


In [3]:
event_df = event_df[event_df['gameweek'].isin([7, 8, 9, 10, 11])]

In [4]:
from helpers.data_handlers import map_coordinates_to_18_zones

field_zones = [map_coordinates_to_18_zones(x, y) for x, y in zip(event_df['x'], event_df['y'])]
end_zone = [map_coordinates_to_18_zones(x, y) for x, y in zip(event_df['endX'], event_df['endY'])]
# add zones to the dataframe
event_df['field_zone'] = field_zones
event_df['end_zone'] = end_zone

In [5]:
event_df.columns

Index(['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'relatedEventId', 'relatedPlayerId', 'blockedX', 'blockedY',
       'goalMouthZ', 'goalMouthY', 'isShot', 'isGoal_x', 'cardType_x',
       'isGoal_y', 'cardType_y', 'x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT',
       'y2_bin_xT', 'start_zone_value_xT', 'end_zone_value_xT', 'xT',
       'teamName', 'shirtNo', 'name', 'position', 'isFirstEleven', 'pro',
       'shortName', 'matchScore', 'simplified_qualifiers', 'gameweek',
       'isOwnGoal_x', 'isOwnGoal_y', 'field_zone', 'end_zone'],
      dtype='object')

In [6]:
# add player names from player_df to event_df
player_names = [player_df[player_df['playerId'] == player_id]['name'].values for player_id in event_df['playerId']]
player_names = [name[0] if len(name) > 0 else 'Unknown' for name in player_names]
event_df['playerName'] = player_names
event_df['simplified_qualifiers'] = [ast.literal_eval(qual) for qual in event_df['simplified_qualifiers'].values]

In [7]:
event_7 = event_df[event_df['gameweek'] == 7]
event_8 = event_df[event_df['gameweek'] == 8]
event_9 = event_df[event_df['gameweek'] == 9]
event_10 = event_df[event_df['gameweek'] == 10]
event_11 = event_df[event_df['gameweek'] == 11]

In [8]:
event_df.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,pro,shortName,matchScore,simplified_qualifiers,gameweek,isOwnGoal_x,isOwnGoal_y,field_zone,end_zone,playerName
87057,2.601095e+09,2,0,0.0,1147,0.0,0.0,0.0,FirstHalf,Start,...,0.000000,NaN,"{'1147': 0, '1730': 0}","{'displayNames': [], 'values': []}",7,NaN,NaN,Out of Play,Out of Play,Unknown
87058,2.601095e+09,2,0,0.0,1730,0.0,0.0,0.0,FirstHalf,Start,...,0.000000,NaN,"{'1147': 0, '1730': 0}","{'displayNames': [], 'values': []}",7,NaN,NaN,Out of Play,Out of Play,Unknown
87059,2.601095e+09,3,0,0.0,1730,50.0,50.0,0.0,FirstHalf,Pass,...,-20.190572,NaN,"{'1147': 0, '1730': 0}","{'displayNames': ['Zone', 'Length', 'PassEndY'...",7,NaN,NaN,Field Zone 9,Field Zone 8,Tim Breithaupt
87060,2.601095e+09,4,0,6.0,1730,34.3,56.6,0.0,FirstHalf,Pass,...,0.000000,NaN,"{'1147': 0, '1730': 0}","{'displayNames': ['Length', 'StandingSave', 'P...",7,NaN,NaN,Field Zone 9,Field Zone 18,Jeffrey Gouweleeuw
87061,2.601096e+09,4,0,28.0,1147,4.7,50.0,0.0,FirstHalf,Pass,...,0.000000,NaN,"{'1147': 0, '1730': 0}","{'displayNames': ['PassEndX', 'Length', 'Angle...",7,NaN,NaN,Field Zone 7,Field Zone 1,Marcel Schuhen


In [9]:
# remove irrelevant columns
irrelevant_columns = ['x1_bin_xT', 'y1_bin_xT', 'x2_bin_xT',
                    'y2_bin_xT', 'start_zone_value_xT', 'end_zone_value_xT', 'xT', 'satisfiedEventsTypes', 'isTouch', 'isOwnGoal_x', 'isOwnGoal_y']

event_df.drop(columns=irrelevant_columns, inplace=True)

In [10]:
all_goals = event_df[event_df['type'] == "Goal"]
all_passes = event_df[event_df['type'] == "Pass"]

From what field zone where the most goals scored during Gameweeks 7 to 11? How many of them were Penalties?

In [11]:
all_goals['field_zone'].value_counts()

field_zone
Field Zone 12    126
Field Zone 11     15
Field Zone 18      5
Field Zone 6       5
Field Zone 5       4
Field Zone 7       4
Field Zone 17      3
Field Zone 9       1
Field Zone 10      1
Field Zone 4       1
Name: count, dtype: int64

In [12]:
from helpers.find_truth import find_events_with_qualifier

penalty = find_events_with_qualifier(all_goals['simplified_qualifiers'], 'Penalty')

np.sum(penalty)

18

From what field zone where the most key passes played that led to a goal? Game Week 9

In [13]:
from helpers.find_truth import find_event_sequence

# pass -> goal
# after a pass event check if there was a goal

assists_9 = find_event_sequence(event_9, 'Pass', 'Goal')

In [14]:
assists_9['field_zone'].value_counts()

field_zone
Field Zone 11    8
Field Zone 6     3
Field Zone 18    3
Field Zone 17    2
Field Zone 12    2
Field Zone 4     1
Field Zone 2     1
Field Zone 5     1
Field Zone 3     1
Name: count, dtype: int64

Which player made the most passes that directly contributet to a goal? Game Week 7-11

In [15]:
# pass -> goal
# after a pass event check if there was a goal

all_assists = find_event_sequence(event_df, 'Pass', 'Goal')

In [16]:
all_assists[["playerName", "playerId"]].value_counts()

playerName        playerId
Leroy Sané        144711.0    6
Fredrik Jensen    322087.0    3
Xavi Simons       399772.0    3
Victor Boniface   384000.0    3
Thomas Müller     37099.0     3
                             ..
Jeremie Frimpong  392644.0    1
Joe Scally        363069.0    1
Julian Ryerson    320436.0    1
Karim Adeyemi     392646.0    1
Álex Grimaldo     107252.0    1
Name: count, Length: 63, dtype: int64

From what field zone did Marvin Ducksch score his freekick against Wolfsburg in Gameweek 10?

In [17]:
event_10[(event_10['type'] == "Goal") * (event_10['playerName'] == "Marvin Ducksch")]

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,pro,shortName,matchScore,simplified_qualifiers,gameweek,isOwnGoal_x,isOwnGoal_y,field_zone,end_zone,playerName
144283,2.613808e+09,42,6,31.0,42,76.1,49.0,6.0,FirstHalf,Goal,...,0.0,NaN,"{'42': 1, '33': 0}","{'displayNames': ['RightFoot', 'DirectFreekick...",10,NaN,NaN,Field Zone 11,Out of Play,Marvin Ducksch


Which player made the most passes that led to an Offside in Gameweek 7?

In [18]:
offside_pass_7 = event_7[(event_7['type'] == "OffsidePass") * (event_7['teamId'] == 37)]   

In [19]:
offside_pass_7

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,pro,shortName,matchScore,simplified_qualifiers,gameweek,isOwnGoal_x,isOwnGoal_y,field_zone,end_zone,playerName
89305,2.602339e+09,515,42,56.0,37,69.1,13.1,42.0,FirstHalf,OffsidePass,...,0.0,NaN,"{'37': 2, '50': 0}","{'displayNames': ['Zone', 'PassEndX', 'GoalDis...",7,NaN,NaN,Field Zone 5,Field Zone 5,Kingsley Coman
89975,2.602398e+09,980,76,6.0,37,93.8,45.1,80.0,SecondHalf,OffsidePass,...,0.0,NaN,"{'37': 2, '50': 0}","{'displayNames': ['Zone', 'GoalDisallowed', 'P...",7,NaN,NaN,Field Zone 12,Field Zone 12,Kingsley Coman
90047,2.602403e+09,1026,80,24.0,37,37.5,38.6,84.0,SecondHalf,OffsidePass,...,0.0,NaN,"{'37': 2, '50': 0}","{'displayNames': ['PlayerCaughtOffside', 'Pass...",7,NaN,NaN,Field Zone 9,Field Zone 10,Noussair Mazraoui


From what field zone did Eintracht Frankfurt score the most Right Foot goals during Game Week 7 to 11?

In [20]:
rightfoot_filter = find_events_with_qualifier(all_goals['simplified_qualifiers'], 'RightFoot')

In [21]:
answer = all_goals[rightfoot_filter * (all_goals['teamId'] == 45)]['field_zone'].value_counts()

From what field zone did Bayern Munich score the most Left Foot goals scored during Gameweek 7 to 11?

In [23]:
leftfoot_filter = find_events_with_qualifier(all_goals['simplified_qualifiers'], 'LeftFoot')

In [24]:
answer = all_goals[leftfoot_filter * (all_goals['teamId'] == 37)]['field_zone'].value_counts()

What player made the most successfull passes into fieldzone 12 during Game Weeks 7 to 11?

In [26]:
end_12 = all_passes[(all_passes['end_zone'] == "Field Zone 12") * (all_passes['outcomeType'] == "Successful")]

In [42]:
end_12['playerName'].value_counts()[:10]

playerName
Leroy Sané             23
Franck Honorat         16
Jonas Hofmann          16
Jan-Niklas Beste       15
Kevin Stöger           11
Leart Paqarada         10
Fredrik Jensen         10
Christopher Trimmel    10
Vincenzo Grifo         10
Tobias Kempe           10
Name: count, dtype: int64

What teams in good form won against teams in bad form in Game Week 9?

In [36]:
league_standings = pd.read_csv('data/Bundesliga/modified/GW7/league_standings.csv')

"In what field zones did Ivan Ordets engage in aerial duels in game week 9?"

In [39]:
event_9[(event_9['type'] == "Aerial") * (event_9['playerName'] == "Ivan Ordets")]

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,pro,shortName,matchScore,simplified_qualifiers,gameweek,isOwnGoal_x,isOwnGoal_y,field_zone,end_zone,playerName
119703,2.609059e+09,5,0,7.0,109,68.3,34.6,0.0,FirstHalf,Aerial,...,0.0,NaN,"{'109': 0, '219': 0}","{'displayNames': ['OppositeRelatedEvent', 'Zon...",9,NaN,NaN,Field Zone 11,Out of Play,Ivan Ordets
119770,2.609064e+09,50,4,17.0,109,49.4,15.2,4.0,FirstHalf,Aerial,...,0.0,NaN,"{'109': 0, '219': 0}","{'displayNames': ['Zone', 'OppositeRelatedEven...",9,NaN,NaN,Field Zone 3,Out of Play,Ivan Ordets


"From what field zones did Harry Kane score his goals in the game Bayern Munich against Borussia Dortmund in game week 10?"

In [41]:
condition = (event_10['type'] == "Goal") * (event_10['playerName'] == "Harry Kane")
event_10[condition]

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,pro,shortName,matchScore,simplified_qualifiers,gameweek,isOwnGoal_x,isOwnGoal_y,field_zone,end_zone,playerName
131306,2.613132e+09,110,8,18.0,37,95.1,47.6,8.0,FirstHalf,Goal,...,0.0,NaN,"{'44': 0, '37': 2}","{'displayNames': ['Assisted', 'IntentionalAssi...",10,NaN,NaN,Field Zone 12,Out of Play,Harry Kane
132476,2.613219e+09,714,71,41.0,37,90.3,46.8,74.0,SecondHalf,Goal,...,0.0,NaN,"{'44': 0, '37': 3}","{'displayNames': ['GoalMouthY', 'BoxCentre', '...",10,NaN,NaN,Field Zone 12,Out of Play,Harry Kane
132779,2.613242e+09,899,92,6.0,37,83.5,50.2,95.0,SecondHalf,Goal,...,0.0,NaN,"{'44': 0, '37': 4}","{'displayNames': ['GoalMouthY', 'Zone', 'Indiv...",10,NaN,NaN,Field Zone 12,Out of Play,Harry Kane
